In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [2]:
ohca_08 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_08.sas7bdat', encoding = 'euc-kr')
ohca_09 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_09.sas7bdat', encoding = 'euc-kr')
ohca_10 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_10.sas7bdat', encoding = 'euc-kr')
ohca_11 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_11.sas7bdat', encoding = 'euc-kr')
ohca_12 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_12.sas7bdat', encoding = 'euc-kr')
ohca_13 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_13.sas7bdat', encoding = 'euc-kr')
ohca_14 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_14.sas7bdat', encoding = 'euc-kr')
ohca_16 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_16.sas7bdat', encoding = 'euc-kr')
ohca_17 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_17.sas7bdat', encoding = 'euc-kr')
ohca_18 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_18.sas7bdat', encoding = 'euc-kr')
ohca_19 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_19.sas7bdat', encoding = 'euc-kr')

In [3]:
ohca_15 = pd.read_table('data\\급성심장정지조사(08-19)SAS\\ohca_15.txt', encoding = 'utf-8', sep='\t')

In [4]:
ohca_15

,NUM,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,...,F_HAPP_ADD_CITY,F_HAPP_ADD_DIST,F_ARREST_DT,F_ARREST_TM,F_CALL_DT,F_CALL_TM,F_AED_EKG,F_DEFIB,F_NO_DEFIB,F_PREH_INS
0,1,2015,11710,서울,송파구,11260.0,서울,중랑구,1,37,...,서울,중랑구,20150101.0,904.0,20150101.0,911.0,50.0,1.0,1.0,3.0
1,2,2015,11140,서울,중구,42190.0,강원,태백시,1,57,...,서울,중구,20140104.0,2059.0,20150104.0,2059.0,50.0,1.0,1.0,1.0
2,3,2015,11140,서울,중구,11590.0,서울,동작구,1,57,...,서울,용산구,20150116.0,1058.0,20150116.0,1058.0,50.0,1.0,1.0,3.0
3,4,2015,11140,서울,중구,11290.0,서울,성북구,1,68,...,서울,중구,20150204.0,1433.0,20150204.0,1433.0,20.0,2.0,NaN,3.0
4,5,2015,11560,서울,영등포구,11560.0,서울,영등포구,2,93,...,서울,영등포구,20150119.0,2320.0,20150119.0,2319.0,50.0,1.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29954,29955,2015,30200,대전,유성구,36110.0,세종,세종시,1,51,...,세종,세종시,99999999.0,9999.0,20151227.0,1744.0,50.0,1.0,1.0,3.0
29955,29956,2015,36110,세종,세종시,36110.0,세종,세종시,2,83,...,세종,세종시,20151219.0,1709.0,20151219.0,1709.0,50.0,1.0,1.0,2.0
29956,29957,2015,36110,세종,세종시,36110.0,세종,세종시,1,93,...,세종,세종시,20151202.0,1210.0,20151202.0,1217.0,50.0,1.0,1.0,1.0
29957,29958,2015,36110,세종,세종시,36110.0,세종,세종시,2,93,...,세종,세종시,99999999.0,9999.0,20151222.0,51.0,50.0,1.0,1.0,3.0


In [5]:
ohca_all = pd.concat([ohca_08, ohca_09, ohca_10, ohca_11, ohca_12, ohca_13, ohca_14, ohca_15, ohca_16, ohca_17, ohca_18, ohca_19], axis = 0)
ohca_all.reset_index(drop = True, inplace = True)
ohca_all

,NUM,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,...,OP8_PRE_VITAL_PR,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED
0,1.0,2008,11410,서울,서대문구,11680,서울,강남구,1,58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,2008,11320,서울,도봉구,11305,서울,강북구,2,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,2008,11110,서울,종로구,11440,서울,마포구,2,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,2008,11590,서울,동작구,11590,서울,동작구,2,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,2008,11560,서울,영등포구,11560,서울,영등포구,1,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324126,30275.0,2019,36110,세종,세종시,36110,세종,세종시,1,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324127,30276.0,2019,43112,충북,청주시 서원구,44230,충남,논산시,1,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324128,30277.0,2019,36110,세종,세종시,36110,세종,세종시,2,56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324129,30278.0,2019,36110,세종,세종시,36110,세종,세종시,2,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
ohca_all.to_csv('data\\급성심장정지조사(08-19)SAS\\ohca_all.csv', encoding = 'euc-kr', index = False)

In [7]:
# 임의의 환자 고유 아이디 생성 (NUM)

ohca_all['NUM'] = np.arange(len(ohca_all))
ohca_all

,NUM,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,...,OP8_PRE_VITAL_PR,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED
0,0,2008,11410,서울,서대문구,11680,서울,강남구,1,58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2008,11320,서울,도봉구,11305,서울,강북구,2,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2008,11110,서울,종로구,11440,서울,마포구,2,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2008,11590,서울,동작구,11590,서울,동작구,2,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2008,11560,서울,영등포구,11560,서울,영등포구,1,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324126,324126,2019,36110,세종,세종시,36110,세종,세종시,1,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324127,324127,2019,43112,충북,청주시 서원구,44230,충남,논산시,1,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324128,324128,2019,36110,세종,세종시,36110,세종,세종시,2,56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324129,324129,2019,36110,세종,세종시,36110,세종,세종시,2,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# ADD_CITY, ADD_DIST 합치기

ohca_all['H_ADD_CITY_DIST'] = ohca_all['H_ADD_CITY'] + ' ' + ohca_all['H_ADD_DIST']
ohca_all['P_ADD_CITY_DIST'] = ohca_all['P_ADD_CITY'] + ' ' + ohca_all['P_ADD_DIST']

In [15]:
ohca_all

,NUM,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,...,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,H_ADD_CITY_DIST,P_ADD_CITY_DIST
0,0,2008,11410,서울,서대문구,11680,서울,강남구,1,58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서대문구,서울 강남구
1,1,2008,11320,서울,도봉구,11305,서울,강북구,2,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 도봉구,서울 강북구
2,2,2008,11110,서울,종로구,11440,서울,마포구,2,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 종로구,서울 마포구
3,3,2008,11590,서울,동작구,11590,서울,동작구,2,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 동작구,서울 동작구
4,4,2008,11560,서울,영등포구,11560,서울,영등포구,1,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구,서울 영등포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324126,324126,2019,36110,세종,세종시,36110,세종,세종시,1,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시,세종 세종시
324127,324127,2019,43112,충북,청주시 서원구,44230,충남,논산시,1,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충북 청주시 서원구,충남 논산시
324128,324128,2019,36110,세종,세종시,36110,세종,세종시,2,56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시,세종 세종시
324129,324129,2019,36110,세종,세종시,36110,세종,세종시,2,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시,세종 세종시


In [16]:
len(ohca_all['PRE_ER_ARREST_DT'].unique())

4476

In [23]:
# 병원 소재지별, 급성심장정지 목격일자별 aggregate

group_hospital = ohca_all.groupby(['PRE_ER_ARREST_DT', 'H_ADMINCODE', 'H_ADD_CITY_DIST'])
group_hospital = pd.DataFrame(group_hospital.size().reset_index(name='count'))

In [24]:
# date column 문자로 바꾸기

group_hospital['PRE_ER_ARREST_DT'] = group_hospital['PRE_ER_ARREST_DT'].astype(str)

In [25]:
# check 

group_hospital

,PRE_ER_ARREST_DT,H_ADMINCODE,H_ADD_CITY_DIST,count
0,20150101.0,11110,서울 종로구,1
1,20150101.0,11200,서울 성동구,1
2,20150101.0,11380,서울 은평구,1
3,20150101.0,11710,서울 송파구,1
4,20150101.0,26470,부산 연제구,1
...,...,...,...,...
120151,99999999,48720,경남 의령군,1
120152,99999999,48740,경남 창녕군,4
120153,99999999,48850,경남 하동군,1
120154,99999999,48890,경남 합천군,1


In [26]:
# date로 sort

group_hospital.sort_values('PRE_ER_ARREST_DT', inplace = True)
group_hospital.reset_index(drop = True, inplace = True)

In [27]:
pd.reset_option('display.max_rows') # 최대 display option reset
group_hospital

,PRE_ER_ARREST_DT,H_ADMINCODE,H_ADD_CITY_DIST,count
0,20080101,42150,강원 강릉시,1
1,20080101,41360,경기 남양주시,1
2,20080101,50110,제주 제주시,1
3,20080101,48870,경남 함양군,1
4,20080101,48250,경남 김해시,1
...,...,...,...,...
120151,99999999,36110,세종 세종시,2
120152,99999999,41115,경기 수원시 팔달구,2
120153,99999999,41250,경기 동두천시,7
120154,99999999,26230,부산 부산진구,5


In [28]:
# 환자 거주지별, 급성심장정지 목격일자별 aggregate

group_patient = ohca_all.groupby(['PRE_ER_ARREST_DT', 'P_ADMINCODE', 'P_ADD_CITY_DIST'])
group_patient = pd.DataFrame(group_patient.size().reset_index(name='count'))

In [29]:
# date column 문자로 바꾸기

group_patient['PRE_ER_ARREST_DT'] = group_patient['PRE_ER_ARREST_DT'].astype(str)

In [30]:
# check 

group_patient

,PRE_ER_ARREST_DT,P_ADMINCODE,P_ADD_CITY_DIST,count
0,20150101.0,11200.0,서울 성동구,1
1,20150101.0,11260.0,서울 중랑구,1
2,20150101.0,11380.0,서울 은평구,1
3,20150101.0,11440.0,서울 마포구,1
4,20150101.0,26470.0,부산 연제구,1
...,...,...,...,...
121898,99999999,48740,경남 창녕군,5
121899,99999999,48850,경남 하동군,1
121900,99999999,48890,경남 합천군,1
121901,99999999,50110,제주 제주시,4


In [31]:
# date로 sort

group_patient.sort_values('PRE_ER_ARREST_DT', inplace = True)
group_patient.reset_index(drop = True, inplace = True)

In [32]:
group_patient

,PRE_ER_ARREST_DT,P_ADMINCODE,P_ADD_CITY_DIST,count
0,20080101,46110,전남 목포시,1
1,20080101,50110,제주 제주시,1
2,20080101,48870,경남 함양군,1
3,20080101,48250,경남 김해시,1
4,20080101,48160,경남 마산시,1
...,...,...,...,...
121898,99999999,41220,경기 평택시,1
121899,99999999,41250,경기 동두천시,3
121900,99999999,41360,경기 남양주시,5
121901,99999999,41480,경기 파주시,1


In [37]:
# 지역코드별 count 몇명인지 한번 check
# group_hospital

hospital_tmp = group_hospital.groupby(['H_ADMINCODE', 'H_ADD_CITY_DIST']).count()
hospital_tmp.sort_values(by='count', ascending = False, inplace = True)
hospital_tmp

,,PRE_ER_ARREST_DT,count
H_ADMINCODE,H_ADD_CITY_DIST,,
41135,경기 성남시 분당구,1924,1924
27200,대구 남구,1563,1563
30140,대전 중구,1555,1555
50110,제주 제주시,1543,1543
41150,경기 의정부시,1475,1475
...,...,...,...
47940,경북 울릉군,1,1
11620,서울 관악구,1,1
45710,전북 완주군,1,1


In [38]:
# 지역코드별 count 몇명인지 한번 check
# group_patient

patient_tmp = group_patient.groupby(['P_ADMINCODE', 'P_ADD_CITY_DIST']).count()
patient_tmp.sort_values(by='count', ascending = False, inplace = True)
patient_tmp

,,PRE_ER_ARREST_DT,count
P_ADMINCODE,P_ADD_CITY_DIST,,
50110,제주 제주시,1308,1308
28237,인천 부평구,1169,1169
11350,서울 노원구,1152,1152
28200,인천 남동구,1101,1101
11290,서울 성북구,1080,1080
...,...,...,...
45110,전북 전주시,1,1
45000,전북 미상,1,1
44000,충남 미상,1,1


In [39]:
# 두 테이블 다 ADMINCODE 숫자로 바꾸기

group_hospital['H_ADMINCODE'] = group_hospital['H_ADMINCODE'].astype(int)
group_patient['P_ADMINCODE'] = group_patient['P_ADMINCODE'].astype(int)

In [40]:
group_hospital

,PRE_ER_ARREST_DT,H_ADMINCODE,H_ADD_CITY_DIST,count
0,20080101,42150,강원 강릉시,1
1,20080101,41360,경기 남양주시,1
2,20080101,50110,제주 제주시,1
3,20080101,48870,경남 함양군,1
4,20080101,48250,경남 김해시,1
...,...,...,...,...
120151,99999999,36110,세종 세종시,2
120152,99999999,41115,경기 수원시 팔달구,2
120153,99999999,41250,경기 동두천시,7
120154,99999999,26230,부산 부산진구,5


In [41]:
group_patient

,PRE_ER_ARREST_DT,P_ADMINCODE,P_ADD_CITY_DIST,count
0,20080101,46110,전남 목포시,1
1,20080101,50110,제주 제주시,1
2,20080101,48870,경남 함양군,1
3,20080101,48250,경남 김해시,1
4,20080101,48160,경남 마산시,1
...,...,...,...,...
121898,99999999,41220,경기 평택시,1
121899,99999999,41250,경기 동두천시,3
121900,99999999,41360,경기 남양주시,5
121901,99999999,41480,경기 파주시,1


In [42]:
idx = pd.read_csv('data\\out of hospital sudden cardiac arrest\\qgis_지도_idx.csv', encoding = 'euc-kr')
idx

,SIGUNGU_CD,SIGUNGU_NM,OBJECTID(air_out_idx),check(수도권=1),location,SOCIAL
0,31370,가평군,1,1,NaN,NaN
1,11230,강남구,2,1,NaN,NaN
2,11250,강동구,3,1,NaN,NaN
3,32030,강릉시,4,0,NaN,NaN
4,11090,강북구,5,1,NaN,NaN
...,...,...,...,...,...,...
235,32310,홍천군,246,0,NaN,NaN
236,31240,화성시,247,1,NaN,NaN
237,36370,화순군,248,0,NaN,NaN
238,32370,화천군,249,0,NaN,NaN


In [43]:
# 지도 idx table이랑 시군구 코드 column 명 맞추기

group_hospital.rename(columns={'H_ADMINCODE' : 'SIGUNGU_CD'}, inplace = True)
group_patient.rename(columns={'P_ADMINCODE' : 'SIGUNGU_CD'}, inplace = True)

In [44]:
# Merge group_hospital with idx with SIGUNGU_CD

group_hospital_idx = pd.merge(group_hospital, idx, on='SIGUNGU_CD', how='inner')
group_hospital_idx.sort_values(by='PRE_ER_ARREST_DT', inplace = True)
group_hospital_idx.reset_index(drop = True, inplace = True)
group_hospital_idx

,PRE_ER_ARREST_DT,SIGUNGU_CD,H_ADD_CITY_DIST,count,SIGUNGU_NM,OBJECTID(air_out_idx),check(수도권=1),location,SOCIAL
0,20080101,31170,울산 동구,1,의왕시,181,1,NaN,NaN
1,20080103,11110,서울 종로구,1,노원구,58,1,NaN,NaN
2,20080104,11110,서울 종로구,1,노원구,58,1,NaN,NaN
3,20080106,31200,울산 북구,1,파주시,231,1,NaN,NaN
4,20080107,31170,울산 동구,1,의왕시,181,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5787,20191230,31170,울산 동구,1,의왕시,181,1,NaN,NaN
5788,20191231,31110,울산 중구,1,과천시,26,1,NaN,NaN
5789,20191231,11170,서울 용산구,1,구로구,35,1,NaN,NaN
5790,99999999,31140,울산 남구,2,오산시,163,1,NaN,NaN


In [45]:
# Merge group_patient with idx with SIGUNGU_CD

group_patient_idx = pd.merge(group_patient, idx, on='SIGUNGU_CD', how='inner')
group_patient_idx.sort_values(by='PRE_ER_ARREST_DT', inplace = True)
group_patient_idx.reset_index(drop = True, inplace = True)
group_patient_idx

,PRE_ER_ARREST_DT,SIGUNGU_CD,P_ADD_CITY_DIST,count,SIGUNGU_NM,OBJECTID(air_out_idx),check(수도권=1),location,SOCIAL
0,20080101,31170,울산 동구,1,의왕시,181,1,NaN,NaN
1,20080102,11230,서울 동대문구,1,강남구,2,1,NaN,NaN
2,20080106,31200,울산 북구,1,파주시,231,1,NaN,NaN
3,20080106,11110,서울 종로구,1,노원구,58,1,NaN,NaN
4,20080107,31170,울산 동구,1,의왕시,181,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4747,20191230,31170,울산 동구,1,의왕시,181,1,NaN,NaN
4748,20191231,11110,서울 종로구,1,노원구,58,1,NaN,NaN
4749,20191231,31110,울산 중구,1,과천시,26,1,NaN,NaN
4750,99999999,31200,울산 북구,1,파주시,231,1,NaN,NaN


In [46]:
group_hospital_idx.isna().sum()

PRE_ER_ARREST_DT            0
SIGUNGU_CD                  0
H_ADD_CITY_DIST             0
count                       0
SIGUNGU_NM                  0
OBJECTID(air_out_idx)       0
check(수도권=1)                0
location                 5792
SOCIAL                   5792
dtype: int64

In [47]:
group_patient_idx.isna().sum()

PRE_ER_ARREST_DT            0
SIGUNGU_CD                  0
P_ADD_CITY_DIST             0
count                       0
SIGUNGU_NM                  0
OBJECTID(air_out_idx)       0
check(수도권=1)                0
location                 4752
SOCIAL                   4752
dtype: int64

In [49]:
#pd.set_option("display.max_rows", 253)
pd.reset_option('display.max_rows')
group_hospital_table = group_hospital_idx.pivot_table(values='count', index=['SIGUNGU_CD', 'SIGUNGU_NM', 'H_ADD_CITY_DIST'], aggfunc=np.sum)
group_hospital_table

,,,count
SIGUNGU_CD,SIGUNGU_NM,H_ADD_CITY_DIST,
11110,노원구,서울 종로구,1270
11140,마포구,서울 중구,600
11170,구로구,서울 용산구,776
11200,동작구,서울 성동구,675
11230,강남구,서울 동대문구,1066
31110,과천시,울산 중구,473
31140,오산시,울산 남구,690
31170,의왕시,울산 동구,554
31200,파주시,울산 북구,288


In [50]:
group_patient_table = group_patient_idx.pivot_table(values='count', index=['SIGUNGU_CD', 'SIGUNGU_NM', 'P_ADD_CITY_DIST'], aggfunc=np.sum)
group_patient_table

,,,count
SIGUNGU_CD,SIGUNGU_NM,P_ADD_CITY_DIST,
11110,노원구,서울 종로구,498
11140,마포구,서울 중구,427
11170,구로구,서울 용산구,648
11200,동작구,서울 성동구,759
11230,강남구,서울 동대문구,1043
31110,과천시,울산 중구,493
31140,오산시,울산 남구,574
31170,의왕시,울산 동구,378
31200,파주시,울산 북구,320


In [46]:
group_patient_idx.to_csv('data\급성심장정지조사(08-19)SAS\group_patient_idx.csv', encoding = 'euc-kr', index = False)
group_hospital_idx.to_csv('data\급성심장정지조사(08-19)SAS\group_hospital_idx.csv', encoding = 'euc-kr', index = False)

In [55]:
group_patient_table.to_csv('data\급성심장정지조사(08-19)SAS\group_patient_table.csv', encoding = 'euc-kr')
group_hospital_table.to_csv('data\급성심장정지조사(08-19)SAS\group_hospital_table.csv', encoding = 'euc-kr')